In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG

%reload_ext autoreload
%autoreload 2

In [4]:
DATA_DIR = CONFIG.CFG.DATA.BASE

In [26]:
import numpy as np
import pandas as pd
import json

In [13]:
train_df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
folds_df = pd.read_csv("folds.csv")
folds_df.head()

,image_id,bbox_count,source,stratify_group,fold
0,00333207f,55,arvalis_1,arvalis_1_3,1
1,005b0d8bb,20,usask_1,usask_1_1,3
2,006a994f7,25,inrae_1,inrae_1_1,1
3,00764ad5d,41,inrae_1,inrae_1_2,0
4,00b5fefed,25,arvalis_3,arvalis_3_1,3


In [14]:
NEW_COLUMNS = ['x', 'y', 'w', 'h']

# separate the bboxes into new columns
sep_bboxes = np.stack(train_df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, new_column in enumerate(NEW_COLUMNS):
    train_df[new_column] = sep_bboxes[:, i]
train_df.drop(columns=['bbox'], inplace=True)

train_df.head()

,image_id,width,height,source,x,y,w,h
0,b6ab77fd7,1024,1024,usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,usask_1,26.0,144.0,124.0,117.0


In [15]:
fold = 0

TRAIN_IDS = folds_df[folds_df['fold'] != fold]['image_id'].unique()
VALIDATION_IDS = folds_df[folds_df['fold'] == fold]['image_id'].unique()

training_df = train_df[train_df['image_id'].isin(TRAIN_IDS)]
validation_df = train_df[train_df['image_id'].isin(VALIDATION_IDS)]

training_df.head()

,image_id,width,height,source,x,y,w,h
0,b6ab77fd7,1024,1024,usask_1,834.0,222.0,56.0,36.0
1,b6ab77fd7,1024,1024,usask_1,226.0,548.0,130.0,58.0
2,b6ab77fd7,1024,1024,usask_1,377.0,504.0,74.0,160.0
3,b6ab77fd7,1024,1024,usask_1,834.0,95.0,109.0,107.0
4,b6ab77fd7,1024,1024,usask_1,26.0,144.0,124.0,117.0


In [3]:
!python3 ../create_folds.py

image_id  ...      h
0  b6ab77fd7  ...   36.0
1  b6ab77fd7  ...   58.0
2  b6ab77fd7  ...  160.0
3  b6ab77fd7  ...  107.0
4  b6ab77fd7  ...  117.0

[5 rows x 8 columns]
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
Created folds.csv with shape (3373, 4)


In [43]:
def create_instances_file(IDS, type="train"):
    out = {
        "info": {
            "description": "",
            "url": "",
            "version": "",
            "year": 2020,
            "contributor": "",
            "date_created": "",
        },
        "licenses": [
            {
                "id": 1,
                "name": None,
                "url": None
            }
        ],
        "categories": [
            {
                "id": 1,
                "name": "wheat",
                "supercategory": "None"
            }
        ],
        "images": [],
        "annotations": []
    }

    k = 0
    for i, image_id in enumerate(IDS):    
        out["images"].append({
            "id": i,
            "file_name": f"{image_id}.jpg",
            "width": 1024,
            "height": 1024,
            "date_captured": "",
            "license": 1,
            "coco_url": "",
            "flickr_url": ""
        })
        
        bboxes = train_df[train_df['image_id'] == image_id][NEW_COLUMNS].values
        areas = bboxes[:, 2] * bboxes[:, 3]
        for j, bbox in enumerate(bboxes):
            out["annotations"].append({
                "id": k,
                "image_id": i,
                "category_id": 1,
                "iscrowd": 0,
                "area": areas[j],
                "bbox": list(bbox),
                "segmentation": []
            })
            k += 1

    with open(f'instances_{type}.json', 'w') as f:
        json.dump(out, f)

In [44]:
create_instances_file(TRAIN_IDS, "train")
create_instances_file(VALIDATION_IDS, "val")